# IMPORTS

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
import logging
import typing
import gc
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
import torch_xla
from torch import nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig, AutoTokenizer, AutoModelForMaskedLM
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

#import torch_xla.core.xla_model as xm
#torch_xla._XLAC._xla_get_all_devices()

# if you are running this notebook from 'home/dev/enefit/notebook'. 
os.chdir('..') # else adjust to point to the root of the project.


In [2]:
from sklearn import metrics
from torch import nn, cuda
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from transformers import AutoTokenizer, AutoModel
from kedro_mlflow.io.metrics import MlflowMetricHistoryDataSet
from kedro_mlflow.io.artifacts import MlflowArtifactDataSet
from kedro_mlflow.io.models import MlflowModelSaverDataSet, MlflowModelLoggerDataSet
from typing import Union
import mlflow
import random
import os
import logging
import torch
import pandas as pd
import transformers as hf
import numpy as np

/home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/__init__.py:44: KedroDeprecationWarning: 'AbstractDataSet' has been renamed to 'AbstractDataset', and the alias will be removed in Kedro 0.19.0
  return getattr(kedro.io.core, name)
/home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/__init__.py:44: KedroDeprecationWarning: 'DataSetError' has been renamed to 'DatasetError', and the alias will be removed in Kedro 0.19.0
  return getattr(kedro.io.core, name)
/home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/__init__.py:44: KedroDeprecationWarning: 'AbstractVersionedDataSet' has been renamed to 'AbstractVersionedDataset', and the alias will be removed in Kedro 0.19.0
  return getattr(kedro.io.core, name)
/home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro_mlflow/io/models/mlflow_model_logger_dataset.py:4: KedroDeprecationWarning: 'DataSetE

In [3]:
%%capture output

%load_ext kedro.ipython
%reload_kedro

if 'output' in locals() and 'error' in output.stderr:
    output.show()

In [4]:
from src.asrsnlp.pipelines.train_bert import nodes

In [5]:
catalog.list()


[
    'train_metrics',
    'eval_metrics',
    'test_data_int',
    'train_data_int',
    'test_data_prm',
    'train_data_prm',
    'test_cleaned_fea',
    'train_cleaned_fea',
    'bertmodel',
    'parameters',
    'params:sub',
    'params:target',
    'params:labels',
    'params:train_params',
    'params:train_params.batch_size',
    'params:train_params.shuffle',
    'params:train_params.num_workers',
    'params:train_sample',
    'params:train_sample.datasize',
    'params:model_params',
    'params:model_params.name',
    'params:model_params.llayer',
    'params:model_params.doratio',
    'params:model_params.max_len',
    'params:model_params.learningrate',
    'params:model_params.epochs',
    'params:eval_params',
    'params:eval_params.batch_size',
    'params:eval_params.shuffle',
    'params:eval_params.num_workers',
    'params:eval_sample',
    'params:eval_sample.datasize'
]

In [6]:
modelname = catalog.load("params:model_params.name")
llayer = catalog.load("params:model_params.llayer")
dropoutratio = catalog.load("params:model_params.doratio")
max_len = 200
trainparams = catalog.load("params:train_params")

                    INFO     Loading data from 'params:model_params.name' (MemoryDataset)...    ]8;id=444844;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=507313;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:model_params.llayer' (MemoryDataset)...  ]8;id=597702;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=157741;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:model_params.doratio' (MemoryDataset)... ]8;id=539124;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=335619;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:model_params.max_len' (MemoryDataset)... ]8;id=604921;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=940915;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:train_params' (MemoryDataset)...         ]8;id=36813;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=472341;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [7]:
model = nodes.ModelClass(modelname, (768,14), dropoutratio)

In [8]:
tokenizer, model = nodes.import_model(modelname, llayer, dropoutratio)

In [9]:
train_feat = catalog.load("train_cleaned_fea")

[11/26/23 15:33:52] INFO     Loading data from 'train_cleaned_fea' (MlflowParquetDataset)...    ]8;id=225602;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=655921;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [25]:
train_loaded = nodes.loader(train_feat, tokenizer, max_len, trainparams, sample=0.0001)

In [26]:
len(train_loaded)

2

In [11]:
loss_fn = nodes.get_loss()

In [12]:
optimizer = nodes.get_optimizer(model, 0.01)

In [13]:
device = nodes.get_device()

In [14]:
ids, mask, token_type_ids, targets = next(iter(train_loaded)).values()

In [15]:
targets.shape

torch.Size([8, 14])

In [ ]:
loss_fn(model(ids, mask, token_type_ids), targets)

In [17]:
def train_model(**kwargs):
    """Train de model.
    kwargs:
        Args:
            mymodel (_type_): _description_
            loss_func (_type_): _description_
            optimizer (_type_): _description_
            epochs (int): _description_
            dataloader (_type_): _description_
            device (_type_): _description_        

    Returns:
        _type_: _description_
    """
    for name, item in kwargs.items():
        if "loss" in name.lower():
            loss_func = torch.nn.BCEWithLogitsLoss()
        if "optim" in name.lower():
            optimizer = item
        if "epoch" in name.lower():
            epochs = item
        if "device" in name.lower():
            device = item
        if isinstance(item, DataLoader):
            dataloader = item
        if isinstance(item, torch.nn.Module):
            modelname = name + "model"
            model = item
    modelsaver = MlflowModelSaverDataSet(filepath=os.path.join("data/06_models",
                                                               modelname),
                                         flavor='mlflow.pytorch')
    epochhistory = MlflowMetricHistoryDataSet(key="training_loss",
                                              save_args={"mode": "list"})
    model.to(device)
    epoch = 1
    losshistory = {}
    while epoch <= epochs:
        with mlflow.start_run(nested=True,
                              run_name=f"epoch_{epoch}"):
            model.train()
            for _, data in enumerate(dataloader, 0):
                ids = data['ids'].to(device, dtype=torch.long)
                mask = data['mask'].to(device, dtype=torch.long)
                token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
                targets = data['targets'].to(device, dtype=torch.float)
                model(ids, mask, token_type_ids)


In [27]:
torch.cuda.empty_cache()

In [28]:
nodes.train_model(mymodel=model, loss=loss_fn, optim=optimizer, epochs=1, dataloader=train_loaded, device=device)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 nodes.train_model(mymodel=model, loss=loss_fn, optim=optimizer, epochs=1, dataloader=tra     │
│   2                                                                                              │
│                                                                                                  │
│ /home/ensai/dev/asrsnlp/src/asrsnlp/pipelines/train_bert/nodes.py:238 in train_model             │
│                                                                                                  │
│   235 │   │   │   │   mask = data['mask'].to(device, dtype=torch.long)                           │
│   236 │   │   │   │   token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)       │
│   237 │   │   │   │   targets = data['targets'].to(device, dtype=torch.float)                    │
│ ❱ 238 │   │   │   │   outputs = model(ids, mask, token_type_ids)                                 │
│   239 │   │   │   │   optimizer.zero_grad()                                                      │
│   240 │   │   │   │   loss = loss_func(outputs, targets)                                         │
│   241 │   │   │   │   losshistory[f"epoch_{epoch}"] = loss.item()                                │
│                                                                                                  │
│ /home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch/nn/module │
│ s/module.py:1518 in _wrapped_call_impl                                                           │
│                                                                                                  │
│   1515 │   │   if self._compiled_call_impl is not None:                                          │
│   1516 │   │   │   return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]        │
│   1517 │   │   else:                                                                             │
│ ❱ 1518 │   │   │   return self._call_impl(*args, **kwargs)                                       │
│   1519 │                                                                                         │
│   1520 │   def _call_impl(self, *args, **kwargs):                                                │
│   1521 │   │   forward_call = (self._slow_forward if torch._C._get_tracing_state() else self.fo  │
│                                                                                                  │
│ /home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch/nn/module │
│ s/module.py:1527 in _call_impl                                                                   │
│                                                                                                  │
│   1524 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1525 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1526 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1527 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1528 │   │                                                                                     │
│   1529 │   │   try:                                                                              │
│   1530 │   │   │   result = None                                                                 │
│                                                                                                  │
│ /home/ensai/dev/asrsnlp/src/asrsnlp/pipelines/train_bert/nodes.py:114 in forward                 │
│                                                            